# Demo: Evaluación de clasificadores

## Objetivos

- Propósito de la evaluación y comparación de clasificadores
- Ventajas e inconvenientes del uso de classification_accuracy como score
- Interpretación de la matriz de confusión
- Métricas computables a partir de la matriz de confusión
- Ajuste jugando con el classification threshold
- Propósito del AUC. Comparación con classification_accuracy

## Classification accuracy
Pima Indians Diabetes dataset originalmente parte del repositorio UCI Machine Learning Repository

In [1]:
# read the data into a pandas DataFrame
import pandas as pd
path = 'pima-indians-diabetes.csv'
col_names = ['pregnant', 'glucose', 'bp', 'skin', 'insulin', 'bmi', 'pedigree', 'age', 'label']
pima = pd.read_csv(path, header=None, names=col_names)

In [2]:
# print the first 5 rows of data
pima.head()

,pregnant,glucose,bp,skin,insulin,bmi,pedigree,age,label
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


Pregunta: ¿Podremos predecir la incidencia de la diabetes a partir de las variables disponibles?

In [3]:
# define X and y
feature_cols = ['pregnant', 'insulin', 'bmi', 'age']
X = pima[feature_cols]
y = pima.label

In [4]:
# split X and y into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=0)

In [5]:
[X_train.shape, X_test.shape]

[(576, 4), (192, 4)]

In [ ]:
# train a logistic regression model on the training set
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

In [ ]:
# make class predictions for the testing set
y_pred_class = logreg.predict(X_test)
y_pred_class

Classification accuracy: porcentaje de predicciones correctas

In [ ]:
# calculate accuracy
from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred_class))

#from sklearn.metrics import accuracy_score
#print(accuracy_score(y_test, y_pred_class))

Null accuracy: accuracy obtenible prediciendo siempre la clase mayoritaria

In [ ]:
# examine the class distribution of the testing set (using a Pandas Series method)
y_test.value_counts()

In [ ]:
# calculate the percentage of ones
y_test.mean()

In [ ]:
# calculate the percentage of zeros
1 - y_test.mean()

In [ ]:
1-(67/(67 + 125))

In [ ]:
# calculate null accuracy (for binary classification problems coded as 0/1)
max(y_test.mean(), 1 - y_test.mean())

Comparamos las diferencias entre predicción y realidad para las 25 primeras muestras de test

In [ ]:
# print the first 25 true and predicted responses
print('True:', y_test.values[0:25])
print('Pred:', y_pred_class[0:25])

### Conclusión
- Classification accuracy es sencillo de entender
- No tiene en cuenta la distribución de los datos
- No informa sobre el tipo de errores que cometemos

## Matriz de confusión

In [ ]:
# IMPORTANT: first argument is true values, second argument is predicted values
print(metrics.confusion_matrix(y_test, y_pred_class))

In [ ]:
metrics.plot_confusion_matrix(logreg, X_test, y_test)

In [ ]:
# save confusion matrix and slice into four pieces
confusion = metrics.confusion_matrix(y_test, y_pred_class)
TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]

In [ ]:
TP

### Cuestión:

¿Qué métricas son las idóneas?

Respuesta:
- Elige según tus objetivos e intereses
- En spam filtering (con spam como clase positiva) optaríamos por precision o specificity porque los falsos negativos son menos malos que los falsos positivos
- En detección de transacciones fraudulentas (con fraude como clase positiva) optaríamos por sensitivity porque los falsos positivos son más aceptables que los falsos negativos

## Ajuste del classification threshold

In [ ]:
# print the first 10 predicted responses
logreg.predict(X_test)[0:10]

In [ ]:
# print the first 10 predicted probabilities for class 1
logreg.predict_proba(X_test)[0:10,1]

In [ ]:
# store the predicted probabilities for class 1
y_pred_prob = logreg.predict_proba(X_test)[:, 1]

In [ ]:
# allow plots to appear in the notebook
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
# histogram of predicted probabilities
plt.hist(y_pred_prob, bins=8)
plt.xlim(0, 1)
plt.title('Histogram of predicted probabilities')
plt.xlabel('Predicted probability of diabetes')
plt.ylabel('Frequency')

Bajando el threshold aumentamos la sensitivity

In [ ]:
# predict diabetes if the predicted probability is greater than 0.3
from sklearn.preprocessing import binarize
y_pred_class = binarize([y_pred_prob], 0.3)[0]

In [ ]:
# print the first 10 predicted probabilities
y_pred_prob[0:10]

In [ ]:
# print the first 10 predicted classes with the lower threshold
y_pred_class[0:10].astype(int)

In [ ]:
# previous confusion matrix (default threshold of 0.5)
print(confusion)

In [ ]:
# new confusion matrix (threshold of 0.3)
print(metrics.confusion_matrix(y_test, y_pred_class))

In [ ]:
# sensitivity has increased (used to be 0.24)
print(46 / float(46 + 16))

In [ ]:
# specificity has decreased (used to be 0.91)
print(80 / float(80 + 50))

Conclusiones (ya conocidas):
- Se utiliza 0.5 como threshold por defecto para transformar probabilidades en predicciones de clases
- Se puede jugar con el threshold para balancear sensitivity y specificity
- Ambas tienen una relación inversa

## Área bajo la curva ROC
Sería interesante saber cómo diferentes thresholds afectan globalmente sin ir probando valores uno a uno

La curva ROC nos da la respuesta

In [ ]:
# IMPORTANT: first argument is true values, second argument is predicted probabilities
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred_prob)
plt.plot(fpr, tpr)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.title('ROC curve for diabetes classifier')
plt.xlabel('False Positive Rate (1 - Specificity)')
plt.ylabel('True Positive Rate (Sensitivity)')
plt.grid(True)

- La curva ROC ayuda a elegir el threshold idóneo para el balance de sensitivity y specificity
- Además podemos ver los pares de valores para los distintos thresholds

In [ ]:
# define a function that accepts a threshold and prints sensitivity and specificity
def evaluate_threshold(threshold):
    print('Sensitivity:', tpr[thresholds > threshold][-1])
    print('Specificity:', 1 - fpr[thresholds > threshold][-1])

In [ ]:
evaluate_threshold(0.5)

In [ ]:
evaluate_threshold(0.31)

El AUC es la proporción que queda por debajo de la curva, es decir, su área

In [ ]:
# IMPORTANT: first argument is true values, second argument is predicted probabilities
print(metrics.roc_auc_score(y_test, y_pred_prob))

- El AUC resume mucha información sobre la bondad del clasificador en un solo valor numérico
- Si elegimos aleatoriamente una muestra de cada clase, el AUC representa la verosimilitud de que el clasificador asigne una probabilidad mayor a la observación de clase positiva
- El AUC es útil aún con datos desbalanceados

In [ ]:
# calculate cross-validated AUC
from sklearn.model_selection import cross_val_score
cross_val_score(logreg, X, y, cv=10, scoring='roc_auc').mean()

GridSearchCV => cogerlo de EjemploClasificacion.ipynb